In [ ]:
input = [] #English
output = [] #Russian

with open('./english-russian.txt', encoding='utf8') as f:
    for line in f:
        sentences = line.split('\t')[:2]
        input.append(sentences[0])
        output.append(sentences[1])

In [ ]:
# check the size of vocab for each language
from collections import Counter
from itertools import chain

english_vocab = []
russian_vocab = []

def get_vocab(li):
  temp = []
  for i in li:
    i = "".join(word for word in i if word not in ('!','.','?')) #other puncutations?
    temp.append(i.split())
  
  return list(chain(*temp))
english_vocab = get_vocab(input)
russian_vocab = get_vocab(output)

print(str(len(input)) + ' English and Russian sentences in the dataset')
print(str(len(Counter(english_vocab).keys())) + ' English words in the dataset')
print(str(len(Counter(russian_vocab).keys())) + ' Russian words in the dataset')

363386 English and Russian sentences in the dataset
21891 English words in the dataset
67443 Russian words in the dataset


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

def tok(sentences):
  t = Tokenizer()
  t.fit_on_texts(sentences)
  return t

def encode(t, len, sentences):
  sequence = t.texts_to_sequences(sentences)
  sequence = pad_sequences(seq, len, padding='post')
  return sequence